# Reenacting the ML model of the 2016 contest winner

The winner team of the 2016 Kaggle-contest used `boosted trees` (**XGBoost**) to achieve their accuracy of `63.88 %` when tested on the secret facies of the `Stuart` and `Crawford` wells. The result is based on the median F1-micro score from 100 realizations of their model.

In this notebook we will try to emulate their results, using their own preprocessing techniques and training both **XGBoost** and **Catboost** models.

Source: https://github.com/seg/2016-ml-contest/blob/master/LA_Team/Facies_classification_LA_TEAM_05_VALIDATION.ipynb

## Package and dataset imports

### Imports for preprocessing and loading dataset

In [5]:
from __future__ import print_function
import numpy as np
import pandas as pd
from statistics import mean
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold , StratifiedKFold
from classification_utilities import display_cm, display_adj_cm
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import LeavePGroupsOut
from sklearn.multiclass import OneVsOneClassifier
from sklearn.ensemble import RandomForestClassifier
from scipy.signal import medfilt

### Imports for training model

In [14]:
from sklearn.ensemble import  RandomForestClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import FunctionTransformer
import xgboost as xgb
from xgboost.sklearn import  XGBClassifier
import catboost
from catboost import CatBoostClassifier

## Loading and preprocessing data

### Loading data and replacing NaN values

In [15]:
# Load data

facies_data = pd.read_csv('./datasets/facies_vectors.csv')

# Parameters

feature_names = ['GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS']
facies_names = ['SS', 'CSiS', 'FSiS', 'SiSh', 'MS', 'WS', 'D', 'PS', 'BS']
facies_colors = ['#F4D03F', '#F5B041','#DC7633','#6E2C00', '#1B4F72','#2E86C1', '#AED6F1', '#A569BD', '#196F3D']

# Store features and labels
X = facies_data[feature_names].values 
y = facies_data['Facies'].values

# Store well labels and depths
well = facies_data['Well Name'].values
depth = facies_data['Depth'].values

# Fill 'PE' missing values with mean
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(X)
X = imp.transform(X)


**Note from LA_team (2016):** *We procceed to run Paolo Bestagini's routine to include a small window of values to acount for the spatial component in the log analysis, as well as the gradient information with respect to depth. This will be our prepared training dataset.*

### Define functions for creating feature window, feature gradient and feature augmentation

In [16]:
# Feature windows concatenation function
def augment_features_window(X, N_neig):
    
    # Parameters
    N_row = X.shape[0]
    N_feat = X.shape[1]

    # Zero padding
    X = np.vstack((np.zeros((N_neig, N_feat)), X, (np.zeros((N_neig, N_feat)))))

    # Loop over windows
    X_aug = np.zeros((N_row, N_feat*(2*N_neig+1)))
    for r in np.arange(N_row)+N_neig:
        this_row = []
        for c in np.arange(-N_neig,N_neig+1):
            this_row = np.hstack((this_row, X[r+c]))
        X_aug[r-N_neig] = this_row

    return X_aug


# Feature gradient computation function
def augment_features_gradient(X, depth):
    
    # Compute features gradient
    d_diff = np.diff(depth).reshape((-1, 1))
    d_diff[d_diff==0] = 0.001
    X_diff = np.diff(X, axis=0)
    X_grad = X_diff / d_diff
        
    # Compensate for last missing value
    X_grad = np.concatenate((X_grad, np.zeros((1, X_grad.shape[1]))))
    
    return X_grad


# Feature augmentation function
def augment_features(X, well, depth, N_neig=1):
    
    # Augment features
    X_aug = np.zeros((X.shape[0], X.shape[1]*(N_neig*2+2)))
    for w in np.unique(well):
        w_idx = np.where(well == w)[0]
        X_aug_win = augment_features_window(X[w_idx, :], N_neig)
        X_aug_grad = augment_features_gradient(X[w_idx, :], depth[w_idx])
        X_aug[w_idx, :] = np.concatenate((X_aug_win, X_aug_grad), axis=1)
    
    # Find padded rows
    padded_rows = np.unique(np.where(X_aug[:, 0:7] == np.zeros((1, 7)))[0])
    
    return X_aug, padded_rows

In [17]:
X_aug, padded_rows = augment_features(X, well, depth)

### Preprocessing

In [18]:
def preprocess():
    
    # Preprocess data to use in model
    X_train_aux = []
    X_test_aux = []
    y_train_aux = []
    y_test_aux = []
    
    # For each data split
    split = split_list[5]
        
    # Remove padded rows
    split_train_no_pad = np.setdiff1d(split['train'], padded_rows)

    # Select training and validation data from current split
    X_tr = X_aug[split_train_no_pad, :]
    X_v = X_aug[split['val'], :]
    y_tr = y[split_train_no_pad]
    y_v = y[split['val']]

    # Select well labels for validation data
    well_v = well[split['val']]

    # Feature normalization
    scaler = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_tr)
    X_tr = scaler.transform(X_tr)
    X_v = scaler.transform(X_v)
        
    X_train_aux.append( X_tr )
    X_test_aux.append( X_v )
    y_train_aux.append( y_tr )
    y_test_aux.append (  y_v )
    
    X_train = np.concatenate( X_train_aux )
    X_test = np.concatenate ( X_test_aux )
    y_train = np.concatenate ( y_train_aux )
    y_test = np.concatenate ( y_test_aux )
    
    return X_train , X_test , y_train , y_test

## Data analysis and model training

### Creating a function to train and test model

In [19]:
# Train and test a classifier

# Pass in the classifier so we can iterate over many seed later.
def train_and_test(X_tr, y_tr, X_v, well_v, clf):
    
    # Feature normalization
    scaler = preprocessing.RobustScaler(quantile_range=(25.0, 75.0)).fit(X_tr)
    X_tr = scaler.transform(X_tr)
    X_v = scaler.transform(X_v)
    
    clf.fit(X_tr, y_tr)
    
    # Test classifier
    y_v_hat = clf.predict(X_v)
    
    # Clean isolated facies for each well
    for w in np.unique(well_v):
        y_v_hat[well_v==w] = medfilt(y_v_hat[well_v==w], kernel_size=5)
    
    return y_v_hat

## Loading blind dataset and making the prediction

In [20]:
#Load testing data
test_data = pd.read_csv('./datasets/validation_data_nofacies.csv')

# Prepare test data
well_ts = test_data['Well Name'].values
depth_ts = test_data['Depth'].values
X_ts = test_data[feature_names].values
    
y_pred = []
print('.' * 10)
for seed in range(10):
    np.random.seed(seed)

    # Make training data.
    X_train, padded_rows = augment_features(X, well, depth)
    y_train = y
    X_train = np.delete(X_train, padded_rows, axis=0)
    y_train = np.delete(y_train, padded_rows, axis=0) 

    # Train classifier  
    clf = make_pipeline(XGBClassifier(learning_rate=0.12,
                                      max_depth=3,
                                      min_child_weight=10,
                                      n_estimators=150,
                                      seed=seed,
                                      colsample_bytree=0.9))

    # Make blind data.
    X_test, _ = augment_features(X_ts, well_ts, depth_ts)

    # Train and test.
    y_ts_hat = train_and_test(X_train, y_train, X_test, well_ts, clf)
    
    # Collect result.
    y_pred.append(y_ts_hat)
    print('|', end='')
    
np.save('TIP160_boosted_100realizations.npy', y_pred)

..........
||||||||||

## Loading labeled test-set and testing accuracy and F1-score

In [21]:
y_pred = np.load('TIP160_boosted_100realizations.npy')

In [22]:
# Creating new dataframe with each of the 100 predicted labels
predicted_df = pd.DataFrame(y_pred).T

# Loading blind dataset with correct facies
target_y = pd.read_csv('./datasets/validation_data_with_facies_new.csv')
target_y = target_y['Facies']

### Appending accuracies to `acc` list and calculating mean

In [23]:
acc = []
f1 = []

print('.'*100)
for i in range(predicted_df.shape[1]):
    tested_acc = accuracy_score(target_y, predicted_df.loc[: , i])
    tested_f1 = f1_score(target_y, predicted_df.loc[:, i], average='micro')
    acc.append(tested_acc)
    f1.append(tested_f1)
    print('|', end='')

mean_acc = mean(acc)
mean_f1 = mean(f1)

....................................................................................................
||||||||||

In [30]:
predicted = pd.DataFrame(y_pred[0], columns=['y_hat'])
predicted['y_target'] = target_y
predicted['hit'] = (predicted['y_hat'] == predicted['y_target']).astype(int)
predicted['adj'] = 0

In [31]:
adjacent_list = [
    [1, 2],
    [1, 2, 3],
    [2, 3],
    [4, 5],
    [4, 5, 6],
    [5, 6, 7, 8],
    [6, 7, 8],
    [6, 7, 8, 9],
    [7, 8, 9]
]

## Determining explicit accuracy and accuracy based on adjasent Facies

In [32]:
# Placeholder list for calculating mean acc of all 100 predicted arrays
adjacent_accuracies = []

# Looping through each of the 100 realizations in y_pred
for column in range(y_pred.shape[0]):
    y_pred_local = y_pred[column]
    adj_accuracy = []
    # Looping through each int in the predicted y-hat array
    for i in range(y_pred_local.shape[0]):
    # Finding the correct Facies category
        for cat in range(1, 10):
            if target_y[i] == cat:
                # If the predicted y-value is one of the adjacent Facies to the actual Facies-class...
                if y_pred_local[i] in adjacent_list[cat-1]:
                    # ... Judge the prediction to be correct
                    adj_accuracy.append(1)
                else:
                    adj_accuracy.append(0)
    # Add accuracy for column to list of all adjacent accuracies
    adjacent_accuracies.append(mean(adj_accuracy))

In [33]:
# Defining the improvement in F1-score since 2016
improvement = (mean_f1 / 0.641 - 1) * 100
# Identifying the mean hit rate of the 100 realizations when considering adjacent Facies
mean_adj_acc = 100*mean(adjacent_accuracies)

In [34]:
print('The F1-micro score, based on 100 realizations of the model, is: {:.3f}.'.format(mean_f1))
print('This is a {:.2f} % improvement on the contest winner F1 score of 0.641.'.format(improvement))
print('The mean accuracy score for adjacent hits is {:.3f}%.'.format(mean_adj_acc))

The F1-micro score, based on 100 realizations of the model, is: 0.686.
This is a 7.02 % improvement on the contest winner F1 score of 0.641.
The mean accuracy score for adjacent hits is 93.169%.


## Creating CSV files with results

### Dataframe with description of predictions

In [29]:
acc_df = pd.DataFrame(acc, columns=['F-1 score'])
adj_acc_df = pd.DataFrame(adjacent_accuracies, columns=['F-1 score'])

In [86]:
described_acc = acc_df.describe()
adj_described_acc = adj_acc_df.describe()

In [89]:
described_acc.to_excel('results1.xlsx')
adj_described_acc.to_excel('results2.xlsx')

### Dataframe with comparison of results

In [81]:
comp_list = {'Mean F1-score': [mean_f1], 
             'LA_team score': [0.641], 
             'Improvement in %': [improvement], 
             'Mean F1-score adjacency': [mean_adj_acc]}
comp_df = pd.DataFrame.from_dict(comp_list)

In [84]:
comp_df.to_excel('comparison.xlsx')

In [19]:
y_pred[1]

array([3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 5,
       8, 8, 8, 8, 8, 8, 6, 6, 6, 6, 4, 4, 4, 4, 4, 4, 4, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 4, 4, 4, 4, 4, 4, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 8, 8, 8, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 8,
       8, 8, 8, 8, 6, 6, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 3, 3, 3, 3,
       3, 3, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 8, 8, 8, 8, 8,
       8, 8, 6, 6, 6, 6, 6, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 3, 3, 3, 8, 8, 8, 8, 8, 8, 8, 8, 8, 6, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 3, 3, 3, 3, 8, 8, 8, 8, 8, 8,